In [1]:
import pandas as pd

In [31]:
"""
Pseudo-code
Read in the monitor file and give the following column names when read in file or create dataframe:
Index,Date,Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Unused,Light_Sensor,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,data_12,data_13,data_14,data_15,data_16,data_17,data_18,data_19,data_20,data_21,data_22,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32

Set variables - LD_start = 06:00, counts_per_day_alive = 100, LD_DD_anlysis = 'LD', start_date = 2/24/2024, end_date = 2/27/2024
Create column called LD_DD with LD_start + 12:00 = LD else DD
Filter on date between start_date & end_date
Filter on LD_DD based on LD_DD_analysis
Create column called date_LD_DD based on concat of date + LD_DD
Group by date_LD_DD and sum each column of data_1 - data_32
Loop through data_1 .. data_32 and determine the alive flies & dead flies - any fly with less than counts_per_day_alive is removed from the analysis - remove from the dataframe and keep number_of_dead_flies
Calculate number of alive flies, number of dead flies, Mean, SD, SEM - stats only on alive flies
Make sure the output is the same as the shiny app
"""

"\nPseudo-code\nRead in the monitor file and give the following column names when read in file or create dataframe:\nIndex,Date,Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Unused,Light_Sensor,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,data_12,data_13,data_14,data_15,data_16,data_17,data_18,data_19,data_20,data_21,data_22,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32\n\nSet variables - LD_start = 06:00, counts_per_day_alive = 100, LD_DD_anlysis = 'LD', start_date = 2/24/2024, end_date = 2/27/2024\nCreate column called LD_DD with LD_start + 12:00 = LD else DD\nFilter on date between start_date & end_date\nFilter on LD_DD based on LD_DD_analysis\nCreate column called date_LD_DD based on concat of date + LD_DD\nGroup by date_LD_DD and sum each column of data_1 - data_32\nLoop through data_1 .. data_32 and determine the alive flies & dead flies - any fly with less than counts_per_day_alive is removed fro

### Set variables - LD_start = 06:00, counts_per_day_alive = 100, LD_DD_anlysis = 'LD', start_date = 2/24/2024, end_date = 2/27/2024


In [2]:
#Set variables - LD_start = 06:00, counts_per_day_alive = 100, LD_DD_anlysis = 'LD', start_date = 2/24/2024, end_date = 2/27/2024
LD_start = '06:00'
counts_per_day_alive = 100
LD_DD_analysis = 'LD'
start_date = '24 Feb 24'
end_date = '27 Feb 24'


### Read in the monitor file and give the following column names when read in file or create dataframe:
### Index,Date,Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Unused,Light_Sensor,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,data_12,data_13,data_14,data_15,data_16,data_17,data_18,data_19,data_20,data_21,data_22,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32

In [3]:
#Read in monitor9.txt, path is /home/twall5/sleep_analysis-3/examples/Monitor9.txt
df = pd.read_csv('/home/twall5/sleep_analysis-3/examples/Monitor9.txt', sep='\t', header=None)

#Give the columns names
#Index,Date,Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Unused,Light_Sensor,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,data_12,data_13,data_14,data_15,data_16,data_17,data_18,data_19,data_20,data_21,data_22,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
df.columns = ['Index','Date','Time','LD-DD','Status','Extras','Monitor_Number','Tube_Number','Data_Type','Light_Sensor','data_1','data_2','data_3','data_4','data_5','data_6','data_7','data_8','data_9','data_10','data_11','data_12','data_13','data_14','data_15','data_16','data_17','data_18','data_19','data_20','data_21','data_22','data_23','data_24','data_25','data_26','data_27','data_28','data_29','data_30','data_31','data_32']

#Combine the date and time collumns, it should look something like '23 Feb 24 11:03:00'
df['Date'] = df['Date'] + ' ' + df['Time']
#Get rid of the time column
df = df.drop(columns=['Time'])
#Rename the collumn to Date_Time
df = df.rename(columns={'Date':'Date_Time'})

#Convert df_copy Date_Time to a datetime object, the format is day Feb 24 time
df['Date_Time'] = pd.to_datetime(df['Date_Time'], format='%d %b %y %H:%M:%S')
df.head(2)

,Index,Date_Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Light_Sensor,data_1,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
0,2,2024-02-23 11:03:00,1,0,9,0,MT,0,1,0,...,28,11,22,8,20,19,29,64,14,37
1,3,2024-02-23 11:04:00,1,0,9,0,MT,0,1,0,...,27,6,25,2,37,23,38,37,10,31


In [4]:
len(df)

7352

### Create column called LD_DD with LD_start + 12:00 = LD else DD

In [5]:
#get the time in the date_time column, if the time is from 06:00 to 18:00 then it is LD, else it is DD
df['LD_DD'] = df['Date_Time'].dt.time
df['LD_DD'] = df['LD_DD'].apply(lambda x: 'LD' if x >= pd.to_datetime(LD_start).time() and x <= pd.to_datetime('18:00').time() else 'DD')

df = df[['Index','Date_Time','LD_DD','LD-DD','Status','Extras','Monitor_Number','Tube_Number','Data_Type','Light_Sensor','data_1','data_2','data_3','data_4','data_5','data_6','data_7','data_8','data_9','data_10','data_11','data_12','data_13','data_14','data_15','data_16','data_17','data_18','data_19','data_20','data_21','data_22','data_23','data_24','data_25','data_26','data_27','data_28','data_29','data_30','data_31','data_32']]
df.head(2)

,Index,Date_Time,LD_DD,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Light_Sensor,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
0,2,2024-02-23 11:03:00,LD,1,0,9,0,MT,0,1,...,28,11,22,8,20,19,29,64,14,37
1,3,2024-02-23 11:04:00,LD,1,0,9,0,MT,0,1,...,27,6,25,2,37,23,38,37,10,31


In [6]:
len(df)

7352

### Filter on date between start_date & end_date

In [50]:
# Adjust end_date to include the entire day of 2024-02-27
#end_date = end_date + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)

# Filter on date between start_date & end_date
#df = df[(df['Date_Time'] >= start_date) & (df['Date_Time'] <= end_date)]
#df

In [7]:
#Make start_date start at 00:00:00 and end_date end at 23:59:59
start_date = start_date + ' 00:00:00'
end_date = end_date + ' 23:59:59'

#Filter on date between start_date & end_date, make sure you include the entire day of end_date
df = df[(df['Date_Time'] >= pd.to_datetime(start_date)) & (df['Date_Time'] <= pd.to_datetime(end_date))]
df.head(2)

,Index,Date_Time,LD_DD,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Light_Sensor,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
777,779,2024-02-24 00:00:00,DD,1,0,9,0,MT,0,0,...,0,4,0,0,0,0,0,0,0,0
778,780,2024-02-24 00:01:00,DD,1,0,9,0,MT,0,0,...,0,4,0,0,0,0,5,0,0,0


In [8]:
len(df)

5760

### Filter on LD_DD based on LD_DD_analysis

In [9]:
#Filtet on LD_DD based on LD_DD_analysis
df = df[df['LD_DD'] == LD_DD_analysis]
df.head(2)

,Index,Date_Time,LD_DD,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Light_Sensor,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
1137,1139,2024-02-24 06:00:00,LD,1,0,9,0,MT,0,0,...,0,0,0,0,0,0,22,11,5,3
1138,1140,2024-02-24 06:01:00,LD,1,0,9,0,MT,0,1,...,21,10,13,0,21,13,24,25,6,12


In [10]:
len(df)

2884

### Create column called date_LD_DD based on concat of date + LD_DD

In [11]:
#Create column called date_LD_DD based on concat of date + LD_DD
df['date_LD_DD'] = df['Date_Time'].dt.date.astype(str) + '_' + df['LD_DD']
#Move date_LD_DD to the right of LD-DD
df = df[['Index','Date_Time','LD_DD','date_LD_DD','LD-DD','Status','Extras','Monitor_Number','Tube_Number','Data_Type','Light_Sensor','data_1','data_2','data_3','data_4','data_5','data_6','data_7','data_8','data_9','data_10','data_11','data_12','data_13','data_14','data_15','data_16','data_17','data_18','data_19','data_20','data_21','data_22','data_23','data_24','data_25','data_26','data_27','data_28','data_29','data_30','data_31','data_32']]
df.head(2)

,Index,Date_Time,LD_DD,date_LD_DD,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
1137,1139,2024-02-24 06:00:00,LD,2024-02-24_LD,1,0,9,0,MT,0,...,0,0,0,0,0,0,22,11,5,3
1138,1140,2024-02-24 06:01:00,LD,2024-02-24_LD,1,0,9,0,MT,0,...,21,10,13,0,21,13,24,25,6,12


In [12]:
# Exclude 'Date_Time' column from the sum operation
df_grouped = df.drop(columns=['Date_Time','Index','LD_DD','LD-DD','Status','Extras','Monitor_Number','Tube_Number','Data_Type','Light_Sensor']).groupby('date_LD_DD').sum()
df_grouped

,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
date_LD_DD,,,,,,,,,,,,,,,,,,,,,
2024-02-24_LD,0,0,9745,3307,6365,6482,6608,7181,10656,7143,...,14340,11659,3414,3231,10023,12283,4664,12056,4435,7603
2024-02-25_LD,0,0,10810,3534,12561,8396,7836,4964,9920,6160,...,11625,11524,5946,364,8037,12141,5817,11777,3307,8448
2024-02-26_LD,0,0,11205,4131,8945,12756,9280,6999,8725,5691,...,10732,16566,10809,142,6894,12879,7517,13155,3615,7101
2024-02-27_LD,0,0,15945,4022,9064,16472,9585,8710,9414,6898,...,11177,21220,12334,17,8592,14291,9347,13491,5013,9338
